In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### 数据集解释：
#### emb_tb_2.csv:用户特征数据集
#### effct_tb.csv:广告点击情况数据集
#### seed_cand_tb.csv:用户类型数据集
#### 本报告主要使用广告点击情况数据，主要涉及字段如下：
#### dmp_id:营销策略编号（源数据文档未做说明，这个根据情况设定为1.对照组，2.营销策略一，3.营销策略二）
#### user_id:支付宝用户id
#### label:用户当天是否点击活动广告(0:未点击，1：点击)

In [3]:
#加载数据
data = pd.read_csv('F:/数据分析/ABtest_data/effect_tb.csv',header = None)
data.columns = ["dt","user_id","label","dmp_id"]
#日志天数属性用户上，删除该列
data = data.drop(columns = "dt")
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [4]:
# 查看计数，平均数，方差，最小值和四分位数，最大值
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


### 重复值处理

In [5]:
data.shape

(2645958, 3)

In [6]:
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

数据行与独立用户数不统一，检查是否存在重复行。

In [7]:
data[data.duplicated(keep = False)].sort_values(by=["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [8]:
#删除重复值
data = data.drop_duplicates()
data[data.duplicated(keep = False)]

,user_id,label,dmp_id


### 3.空值处理

In [9]:
data.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2632975 non-null  int64
 1   label    2632975 non-null  int64
 2   dmp_id   2632975 non-null  int64
dtypes: int64(3)
memory usage: 80.4 MB


In [10]:
#### 数据集没有空值可以不用处理

### 4.异常值检查
##### 通过透视表格检查各属性字段是否存在不合理取值

In [11]:
data.pivot_table(index = "dmp_id",columns = "label",values = "user_id",aggfunc = "count",margins = True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


属性字段没有发现一场自，无需进行处理

### 5.数据类型

In [12]:
data.dtypes

user_id    int64
label      int64
dmp_id     int64
dtype: object

数据类型正常不需要转换

### 2.2样本容量检验
一般采用样本量计算工具：sample size calculator.使用可以看自己写的ABtest学习文档

In [13]:
data[data["dmp_id"] == 1]["label"].mean()

0.012551012429794775

这个数字是对照组的点击率，为1.26%，我们需要的新的营销策略能让广告点击率至少提高1个百分点，那么
我们在网站中https://www.evanmiller.org/ab-testing/sample-size.html
    baseline conversion rate 框中输入1.26
    minimum detectable effect 框中输入 1
    计算得出我们实验所需要的的最小样本量为2167

In [14]:
#查看每个组中的样本数
data["dmp_id"].value_counts()

1    1905663
2     411107
3     316205
Name: dmp_id, dtype: int64

可以看到三种策略的样本数都比我们最小样本数量大，因此样本合适

In [15]:
#保存清洗好的数据备用
data.to_csv("F:/数据分析/ABtest_data/output.csv",index = False)

### 3.假设验证
先观察几组实验的点击情况

In [16]:
print("对照组：",data[data["dmp_id"] ==1]["label"].mean())
print("策略一组：",data[data["dmp_id"] ==2]["label"].mean())
print("策略二组：",data[data["dmp_id"] ==3]["label"].mean())

对照组： 0.012551012429794775
策略一组： 0.015314747742072015
策略二组： 0.026191869198779274


从点击率来看，策略一和策略二在对照组的基础上都有一定的提升。
其中策略一提高了0.2个百分点，策略二提高了1.3个百分点，只有策略二满足了我们对点击率提升最小值的要求
接下来需要进行假设验证，来看看策略二的点击率提升是否显著

#### a.零假设和责备假设
##### 记对照组点击率为p1，策略二点击率为p2，则：
零假设设为H0：p1>=p2
备择假设为H1：p1< p2
##### b.分布类型，检验类型和显著性水平
样本服从两点分布，独立双样本，样本大小为n>30，总体均值和标准差未知，所以采用z检验，显著性水平取0.05

#### 3.1公式计算

In [20]:
#用户数
n_old = len(data[data.dmp_id == 1]) #对照组
n_new = len(data[data.dmp_id == 3]) #策略二

#点击数
c_old = len(data[data.dmp_id == 1][data.label == 1])
c_new = len(data[data.dmp_id == 3][data.label == 1])

#计算点击率
try:
    if c_new == 0:
        print("不做运算")
    else:
        r_old = c_old / n_old
        r_new = c_new / c_new
except:
    print("除数为0")
#总和点击率
r = (c_old + c_new) / (n_old + n_new)
print("总和点击率：", r)

总和点击率： 0.014492310074225832


C:\Users\18042\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\18042\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [22]:
#计算检验统计量Z
z = (r_old - r_new) / np.sqrt(r*(1-r)*(1/n_old + 1/n_new))
print("检验统计量z:", z)

检验统计量z: -4302.928619508961


In [23]:
#看显著水平0.05对应的Z的分位数
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
z_alpha

-1.6448536269514729

z_aloha = -1.644，检验统计量z为-4032 ,该检验为左侧单尾检验，拒绝域为{z<z_alpha}.所以我们可以得出结论，原假设不成立，策略二点击率的提升在统计上是显著的

In [24]:
import statsmodels.stats.proportion as sp
z_score, p = sp.proportions_ztest([c_old, c_new],[n_old, n_new], alternative = "smaller")
print("检验统计量z：",z_score,"，p值：", p)

检验统计量z： -59.44168632985996 ，p值： 0.0


p值约等于0.p<显著值，所以结论一样，拒绝原假设。
综上所述，两种营销策略中，策略二对广告点击率有显著提升效果，且相较于对照组点击率提升了近一倍，因而在两组营销策略中应选择第二组进行推广